<a href="https://colab.research.google.com/github/S18-Niloy/Tensorflow_tutorial/blob/main/flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive

In [ ]:
from __future__ import absolute_import, division, print_function,unicode_literals

import tensorflow as tf

import pandas as pd

specific dataset seperates flowers into 3 different classes of species.

               Setosa
               Versicolor
               Virginica
The information about each flower is the following.

               sepal length
               sepal width
               petal length
               petal width

In [ ]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
CSV_COLUMNS = ['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versicolor','Virginica']

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/flower/iris_training.csv',names = CSV_COLUMNS, header = 0)
test = pd.read_csv('/content/gdrive/My Drive/flower/iris_test.csv',names = CSV_COLUMNS, header = 0)

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Popping the species column of and use it as a label

In [ ]:
ytrain = train.pop('Species')
ytest = test.pop('Species')
train.head() #species column is gone

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape

(120, 4)

In [ ]:
ytrain.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

**TensorFlow estimators work with input functions. The signature of an input function returns a tuple of features and labels. Features are a dictionary of feature names and numeric value arrays. Labels are an array of values.**

**The batch size is a number of samples processed before the model is updated. The number of epochs is the number of complete passes through the training dataset. The size of a batch must be more than or equal to one and less than or equal to the number of samples in the training dataset.**

**from_tensor_slices() method, we can get the slices of an array in the form of objects by using tf.data.Dataset.from_tensor_slices() method.**

**repeat() method in TensorFlow. repeat() method of tf. data. Dataset class is used for repeating the tensors for a given count times in dataset.**

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

**Feature column describes how to use an input**

**feature_column. numeric_column('b') Feature column describe a set of transformations to the inputs.**

In [ ]:
my_feature_columns = []

for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


**Estimator module just store a bunch of pre made models from tensorflow**

**DNNClassifier is used to pass our feature columns just like linear classifier**

**Hidden units essentially helps us building the architecture of neural network. Middle layers are called hidden layers**

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30, 10],
    n_classes=3)

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, ytrain, training=True),
    steps=5000)

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, ytest, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.867



In [ ]:
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

Please type numeric values as prompted.
SepalLength: 2.4
SepalWidth: 3.3
PetalLength: 4.5
PetalWidth: 2.4
Prediction is "Virginica" (56.9%)
